# Create the GC dataset

Collect the data reduced by G. Chen into two Pandas DataFrames, and add the auxiliary data taken from the file headers. The light curves are found from `data/gtc_gc` as a set of csv files for each night and passband.

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from src.core import *
dfas = [pd.read_hdf('data/aux.h5','night1'),
        pd.read_hdf('data/aux.h5','night2')]

In [3]:
pb_keys = 'nb k na pr'.split()
files_per_night = [dict(
    nb  = sorted(glob('data/gtc_gc/n{:d}nb/*.dat'.format(i))),
    k   = sorted(glob('data/gtc_gc/n{:d}k/*.dat'.format(i))),
    na  = sorted(glob('data/gtc_gc/n{:d}na/*.dat'.format(i))),
    pra = sorted(glob('data/gtc_gc/n{:d}c50a/*.dat'.format(i))),
    prb = sorted(glob('data/gtc_gc/n{:d}c50b/*.dat'.format(i))))
    for i in range(1,3)]

In [4]:
for i,fpn in enumerate(files_per_night):
    fpn['pr'] = list(concatenate([[fa,fb] for fa,fb in zip(fpn['pra'], fpn['prb'])]))
    del fpn['pra'], fpn['prb']

In [5]:
data = [[[loadtxt(f) for f in files[pb]] for pb in pb_keys] for files in files_per_night]

In [6]:
dfs = [pd.DataFrame(d[0][0][:,[0,5,6,8,9,10]], 
                    columns='bjd x y fwhm airmass rotang'.split()) 
       for d in data]

In [7]:
for df,dfa,d in zip(dfs,dfas,data):
    for pb,pbd in zip('nb Na K pr'.split(), d):
        for i,d in enumerate(pbd):
            if i != 12 or pb in ['Na', 'K', 'pr']:
                df['target_{:s}{:02d}'.format(pb,i+1)] = d[:,1]
                df['comparison_{:s}{:02d}'.format(pb,i+1)] = d[:,3]
                df['relative_{:s}{:02d}'.format(pb,i+1)] = d[:,1] / d[:,3]

In [8]:
dfs[0].index = dfas[0].index
dfs[1].index = dfas[1].index[1:]

dfs = [pd.merge(dfn, dfa, left_index=True, right_index=True, suffixes=('','_aux'))
       for dfn,dfa in zip(dfs,dfas)]

In [9]:
cols = [c for c in dfs[0].columns if 'relative_pr' in c]

In [10]:
dfs[0].to_hdf(join(DRESULT,'gtc_light_curves_gc.h5'), 'night1')
dfs[1].to_hdf(join(DRESULT,'gtc_light_curves_gc.h5'), 'night2')

---

<center> &copy; 2017 Hannu Parviainen</center>